Exploratory data analysis using ANOVA:

In [178]:
import scipy.stats as stats

diabetic = X[:, 17 - 1] == 1
stats.f_oneway(Y[diabetic], Y[np.logical_not(diabetic)])

F_onewayResult(statistic=2.7849569187612762, pvalue=0.096270975783822321)

ANOVA indicates a p-value of 0.09, a little promising. Now let's try basic logistic regression:

In [180]:
from sklearn import linear_model
from sklearn.model_selection import KFold


kf.get_n_splits(X)
print(kf)

def kfold_validate (X_sub, k):
    kf = KFold(n_splits = k)
    sum = 0
    for train, test in kf.split(X_sub):
        reg = linear_model.LogisticRegression()
        reg.fit(X_sub[train], Y[train])
        sum += (reg.score(X_sub[test], Y[test]))
    return sum / k
        
def filter_select(X, Y, num_features):
    # ignore the bad columns
    avail_features = list(range(0, 75))
    for garbage_col in columns_to_exclude:
        avail_features.remove(garbage_col)
        
    included_features = [] #[17 - 1] # diabetes    
    for i in range(num_features):
        best_feature = -1
        best_score = -1
        for feature in avail_features:
            score = kfold_validate(X[:,included_features + [feature]], 4)
            if score > best_score:
                best_score = score
                best_feature = feature
        included_features += [best_feature]
        avail_features.remove(best_feature)
        print(included_features, best_score)
    return included_features
            
filter_select(X, Y, 10)
#for i in range(76):
#    reg = linear_model.LogisticRegression()
#    reg.fit(X, Y)
#    reg.coef_

KFold(n_splits=4, random_state=None, shuffle=False)
[50] 0.758400402414
[50, 8] 0.776106639839
[50, 8, 31] 0.8046277666
[50, 8, 31, 43] 0.829426559356
[50, 8, 31, 43, 28] 0.836519114688
[50, 8, 31, 43, 28, 21] 0.843661971831
[50, 8, 31, 43, 28, 21, 38] 0.843661971831
[50, 8, 31, 43, 28, 21, 38, 34] 0.843712273642
[50, 8, 31, 43, 28, 21, 38, 34, 23] 0.84014084507
[50, 8, 31, 43, 28, 21, 38, 34, 23, 56] 0.84014084507


[50, 8, 31, 43, 28, 21, 38, 34, 23, 56]

In [146]:
np.mean(X[Y==1, 50]),np.mean(X[Y==0, 50])

(5.6879999999999997, 3.7006369426751591)

In [ ]:
from scipy.stats import linregress

